# Question 3

In [1]:
using Plots, DataFrames, CSV, GLM
using Optim, Distributions, Random, ForwardDiff
using LinearAlgebra, StatsFuns, FixedEffectModels
using LaTeXTabulars

In [5]:
df = DataFrame(CSV.File("../data/ps1_ex3.csv"));

# I need to get the 0 share in each market 
gdf = groupby(df, :market)
gdf = combine(gdf, :Shares => sum)
gdf[!,:shares_0] = 1 .- gdf[!,:Shares_sum];
df = leftjoin(df, gdf, on=:market);

# Will need ln(s_jt / s_0t) and ln(s_jt)
df[!,:log_sj_s0] = log.(df[!,:Shares]./df[!,:shares_0]);
df[!,:log_sj] = log.(df[!,:Shares]);

# Part 1: Estimating $(\alpha, \beta)$

We have the moment condition $E[\xi_{jt} | z_{jt}] = 0$, hence we can estimate the following with TSLS: 

$$ ln\left(\frac{s_{jt}}{s_{0t}}\right) = - \alpha p_{jt} + x_{jt}' \beta + \xi_{jt}$$

In [6]:
estimates = reg(df, @formula(log_sj_s0 ~ x + (Prices ~ z)))
α = -coef(estimates)[3];
β = coef(estimates)[2];
df[!,:ξ] = (df[!,:log_sj_s0] - predict(estimates,df)) .+ coef(estimates)[1];

# Parameters
print("α: $α", "\nβ: $β")

# Brand preference
gdf = groupby(df, :Product)
gdf = combine(gdf, :ξ => mean)

α: 0.4674770969358872
β: 0.3046723858039954

,Product,ξ_mean
,Int64,Float64
1,1,1.38619
2,2,1.39275
3,3,0.401759
4,4,0.394211
5,5,0.399346
6,6,0.39907


In [18]:
# output to table
latex_tabular("output/ps1_q3_estimates.tex",
              Tabular("cc"),
              [Rule(:top),
              ["Parameter", "Estimates"],
              ["\$\\alpha\$" round(α, digits=3)],
              ["\$\\beta\$", round(β, digits=3)],
              ["\$\\xi_1\$", round(gdf[1,:ξ_mean], digits=3)], 
              ["\$\\xi_2\$", round(gdf[2,:ξ_mean], digits=3)],
              ["\$\\xi_3\$", round(gdf[3,:ξ_mean], digits=3)],
              ["\$\\xi_4\$", round(gdf[4,:ξ_mean], digits=3)],
              ["\$\\xi_5\$", round(gdf[5,:ξ_mean], digits=3)],
              ["\$\\xi_6\$", round(gdf[6,:ξ_mean], digits=3)],
               Rule(:bottom)])

# Part 2: Finding price elasticities

$$\eta_{jkt} = \frac{\partial s_{jt}}{\partial p_{kt}} \frac{p_{kt}}{s_{jt}} = \begin{cases}
        - \alpha p_{jt} (1-s_{jt}) & j=k \\
        \alpha p_{kt} s_{kt} & j\neq k
    \end{cases}$$

In [19]:
# reshape dataframe of prices and shares
share_array = Array(unstack(df, :market, :Product, :Shares)[!,2:7]);
price_array = Array(unstack(df, :market, :Product, :Prices)[!,2:7]);

In [23]:
# for each market, we should get a JxJ matrix 
T = size(share_array)[1]
J = size(share_array)[2]

# for each market, calculate each elasticities and store market substitution matrix ηₜ in vector η
η = Matrix{Float64}[] 
for t in 1:T
    ηₜ = zeros(Float64,J,J)
    for j in 1:J
        for k in 1:J
            if j == k
                ηₜ[j,k] = - α * price_array[t,j] * (1 - share_array[t,j]) 
            else
                ηₜ[j,k] = α * price_array[t,k] * share_array[t,k]
            end
        end
    end
    push!(η,ηₜ)
end

In [24]:
latex_tabular("output/ps1_q3_elasticities.tex",
              Tabular("cccccc"),
              [Rule(:top),
               round.(mean(η), digits=3),
               Rule(:bottom)])

# Part 3: Recover marginal costs
Assuming that each product-market is a different firm, we can obtain the following from the FOC from the firm optimization:
$$ s_{jt}(p) + (p_{jt} - mc_{jt}) \frac{\partial s_{jt}}{\partial p_{kt}} = 0 $$
which implies:
$$ s_{jt}(p) \frac{1}{\frac{\partial s_{jt}}{\partial p_{kt}}} + p_{jt} = mc_{jt} $$

We can equivalently write this as:
$$ mc_{jt} = p_{jt} - \frac{1}{\alpha}\frac{1}{(1-s_{jt})}$$
which makes it clear that average marginal costs can be related to prices and shares directly from the data given an estimate of $\alpha$.

In [39]:
df[!,:invshares] = 1 ./ (1 .- df[!,:Shares]);

df[!,:mc] = df[!,:Prices] - (1/α) .* df[!,:invshares];

means = groupby(df, :Product)
means = Array(combine(means, [:mc,:Prices, :Shares] .=> mean))

6×4 Matrix{Float64}:
 1.0  0.667126  3.35995  0.202451
 2.0  0.671897  3.36753  0.203076
 3.0  0.678684  3.03306  0.0903493
 4.0  0.688906  3.03977  0.0889407
 5.0  0.682632  3.03103  0.0881716
 6.0  0.682689  3.03815  0.0906875

In [40]:
latex_tabular("output/ps1_q3_marginal_cost.tex",
              Tabular("cccc"),
              [Rule(:top),
              ["Product", "Mean MC", "Mean Prices", "Mean Share"],
              Rule(),
               round.(means, digits=3),
               Rule(:bottom)])

# Part 4: Product 1 exits the market
To simulate $j = 1$ leaving the market, we solve for the market prices that minimize the firms' first-order conditions as the objective, subject to demand (expressed as shares) as estimated above. We can use the fact that $\frac{\partial s_{k}}{\partial p_{k}} = -\alpha s_k (1-s_k)$. We can then plug the prices into the share equation to get simulated market shares. That is:
$$ \min_{p} || FOC_k ||  = || s(p) + \alpha (p - c)s(p)(1 - s(p)) || $$
$$ \text{s.t. } s_{j}(p) = \frac{\exp(-\alpha p_{j} + x_{j}'\beta + \xi_{j})}{1 + \sum_{k = 2}^5 \exp(-\alpha p_{k} + x_{k}'\beta + \xi_{k})}$$ 
Following Conlon and Gortmaker (2020), we can solve this with a simple fixed point algorithm, starting with a candidate vector of market prices, $p_k^{(i)}$, then calculating the implied price from the FOC by $$p_k^{(i+1)} = c + \frac{1}{\alpha} + s(p_k^{(i)})(p_k^{(i)} - c_k)$$ until some tolerance for $| p_k^{(i+1)} - p_k^{(i)} |$.

In [42]:
function implied_shares(x,ξ,α,β,p) # calculate market demand given parameters, prices, and products
    J = size(p)
    
    # numerator
    u = zeros(J);
    @. u = exp(-α * p + x * β + ξ);
    
    # denominator
    u_sum = sum(u) + 1;
    
    # calculate shares
    s = u ./ u_sum;
    
    return s
end

function iterate_price(x,ξ,α,β,c,p) # ## Conlon and Gortmaker (2020) fixed point iteration for price equilibrium
    
    s = implied_shares(x,ξ,α,β,p)
    # p = c + 1/α .* (1 ./ (1 .- s))
    p = c .+ 1/α .+ s .* (p - c)   
    
    return p
end

function foc(α,s,c,p) # firm objective FOC
    J = size(p)
    
    # calculate firm FOCs
    foc = zeros(J)
    @. foc = s + (α * (p - c) * s * (1 - s)); 
    
    return foc
end

function objective(x,ξ,α,β,c,p) # evaluate L2 distance of FOCs
    # number of parameters
    J = size(p)
    
    s = implied_shares(x,ξ,α,β,p)
    firmfoc = foc(α,s,c,p)
    
    # objective
    F = sum(firmfoc.^2)
    
    return F
end

objective (generic function with 1 method)

In [43]:
# Define counterfactual
simulation = df[!,[:market, :Product, :x, :ξ, :mc, :Prices, :Shares]];
simulation = simulation[(df.Product .!= 1), :];

# control loop
nmkts = 1000
tol = 1e-16
max_iter = 1000

# Solve for prices, looping through each market
prices = Float64[]; 
shares = Float64[];
focs = Float64[];   # L2-norm of firm focs
fail = Bool[]       # reaches max_iter

n_params = size(simulation[(simulation.market .== 1), :])[1]; 

count = 0
@time for mkt in 1:nmkts
    sim_mkt = simulation[(simulation.market .== mkt), :];
    
    # start somewhere close to true prices
    p₀ = Array(sim_mkt[!,:Prices]) 
    
    # define inputs
    x = Array(sim_mkt[!,:x]);
    ξ = Array(sim_mkt[!,:ξ]);
    c = Array(sim_mkt[!,:mc]);
    
    # fixed point iteration
    pᵢ = p₀
    ε = 1
    i = 0
    while (ε > tol) && (i < max_iter)   
        i += 1
        pᵢ₊₁ = iterate_price(x,ξ,α,β,c,pᵢ)
        ε = maximum(abs.(pᵢ₊₁ - pᵢ))
        pᵢ = max.(pᵢ₊₁,zeros(n_params))
    end
    
    # store results   
        #println("mkt: $mkt", " ε: $ε", " i: $i")
        #println("p: $pᵢ")  
    append!(prices,pᵢ)
    append!(focs,objective(x,ξ,α,β,c,pᵢ))
    append!(shares,implied_shares(x,ξ,α,β,pᵢ))
    append!(fail,(i == 1000))
end

simulation[!,:Sim_Prices] = prices;
simulation[!,:Sim_Shares] = shares;

print("Maxed out iterations: ", sum(fail), "\nMax L2-FOC distance: ", maximum(focs))

  0.974060 seconds (5.36 M allocations: 313.945 MiB, 8.39% gc time, 50.87% compilation time)
Maxed out iterations: 233
Max L2-FOC distance: 0.7974855878851073

In [47]:
# Get original prices and shares
old_results = groupby(df, :Product);
old_results = combine(old_results, [:Prices, :Shares] .=> mean);
rename!(old_results, :Prices_mean => :Old_Prices, :Shares_mean => :Old_Shares);

# Add simulated prices and shares
sim_results = groupby(simulation, :Product);
sim_results = combine(sim_results, [:Sim_Prices, :Sim_Shares] .=> mean);

# Combine with old results, old market now 0
sort!(push!(sim_results, [1,0,0]),:Product)

,Product,Sim_Prices_mean,Sim_Shares_mean
,Int64,Float64,Float64
1,1,0.0,0.0
2,2,3.518,0.244264
3,3,3.09522,0.113266
4,4,3.10046,0.111395
5,5,3.09185,0.110741
6,6,3.10044,0.113597


In [49]:
res = Array(innerjoin(old_results,sim_results, on = :Product))
latex_tabular("output/ps1_q3_counterfactual_prices.tex",
              Tabular("ccccc"),
              [Rule(:top),
              ["Product", "Old Price", "Old Share", "Counterfactual Price", "Counterfactual Share"],
              Rule(),
               round.(res, digits=3),
               Rule(:bottom)])

# Part 5: Profits and welfare
Profit for each firm is: $$ \pi_{jt} = s_{jt}(p_{jt} - c_{jt})$$
Average change in industry profit is then: $$ \Delta \bar{\pi} = \frac{1}{T} \sum_{t = 1}^{1000} \left( \sum_{j = 2}^5 s_{jt}' (p_{jt}' - c_{jt}) - \sum_{j = 1}^5 s_{jt} (p_{jt} - c_{jt}) \right) $$
Market welfare (where $\gamma \approx 0.577$ is the Euler constant) is given by: $$ W_t = E\left[\max_j u_{ijt}\right] = \alpha E[y_i] + \gamma - \ln(s_{0t}) $$
Average change in market welfare is then the average log odds ratio of consuming the outside option: $$ \Delta \bar{W} = \frac{1}{T} \sum_{t = 1}^{1000} \ln\left(\frac{s_{0t}}{s_{0t}'}\right)$$

In [65]:
## Calculate profits
# Individual firm profit
df[!,:Profit]             = df[!,:Shares] .* (df[!,:Prices] .- df[!,:mc]);
simulation[!,:Sim_Profit] = simulation[!,:Sim_Shares] .* (simulation[!,:Sim_Prices] .- simulation[!,:mc]);

# Pull total profits in each market pre- and post-simulation
gdf = groupby(df,:market);
gdf = combine(gdf, [:Profit] => sum);

gsim = groupby(simulation,:market);
gsim = combine(gsim, [:Sim_Profit] => sum)

# Average change in market profits
profits = innerjoin(gdf, gsim, on = :market);
profits[!,:change] = profits[!,:Sim_Profit_sum] .- profits[!,:Profit_sum];
Δ̄π = round(mean(profits[!,:change]); digits = 2);
pos_markets = profits[profits[!,:change].>0, :market]

println("Average change in profits: \$ $Δ̄π")
println("Markets that has positive change in profits: $pos_markets")

Average change in profits: $ -0.16
Markets that has positive change in profits: [282, 525, 672, 808]


In [210]:
## Calculate welfare
# Outside good shares pre- and post-simulation
gdf = groupby(df, :market);
gdf = combine(gdf, :Shares => sum);
gdf[!,:shares_0] = 1 .- gdf[!,:Shares_sum];

gsim = groupby(simulation, :market);
gsim = combine(gsim, :Sim_Shares => sum);
gsim[!,:shares_0] = 1 .- gsim[!,:Sim_Shares_sum];

# Compute welfare, and convert to $ by dividing by MU of income (α)
Δ̄W = mean(log.(gdf[!,:shares_0]) .- log.(gsim[!,:shares_0])) 
dollar_Δ̄W = round(Δ̄W / α; digits = 2)

println("Average change in welfare: $Δ̄W (\$ $dollar_Δ̄W)")

Average change in welfare: -0.26118808227352497 ($ -0.56)
